In [ ]:
# import nltk
# from nltk.corpus import stopwords
# #nltk.download('stopwords')
# from nltk.stem import WordNetLemmatizer, SnowballStemmer
# from nltk.stem.porter import *
# #nltk.download('wordnet')
# from nltk import PorterStemmer as stemmer
# stemmer = stemmer()

In [6]:
import pandas as pd
import os
import re
import numpy as np
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Proves de dues llibreries

In [17]:
#Textblob
opinion = TextBlob("I am fabulous")
opinion.sentiment

Sentiment(polarity=0.4, subjectivity=1.0)

In [22]:
opinion = TextBlob("I am good")
opinion.sentiment

Sentiment(polarity=0.7, subjectivity=0.6000000000000001)

In [16]:
#Vader
analyser = SentimentIntensityAnalyzer()

In [20]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [21]:
sentiment_analyzer_scores("I am fabulous")

I am fabulous--------------------------- {'neg': 0.0, 'neu': 0.37, 'pos': 0.63, 'compound': 0.5267}


In [23]:
sentiment_analyzer_scores("I am good")

I am good------------------------------- {'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.4404}


### Importar dataset i fer append del train i el test datasets

In [77]:
dataset = pd.read_csv('database_lyrics.csv', sep = ";")

In [78]:
dataset

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0
...,...,...,...,...,...
19045327,TRZZZZD128F4236844,2466899,easili,1,1
19045328,TRZZZZD128F4236844,2466899,disast,1,1
19045329,TRZZZZD128F4236844,2466899,frown,1,1
19045330,TRZZZZD128F4236844,2466899,teas,1,1


In [172]:
df= dataset.loc[:,['track_id', 'word', 'count']]

In [173]:
prueba= df[60:80]

In [174]:
#Add dictionary
lmdict = {'Negative': ['slice', 'devast', 'element'], 'Positive': ['and', 'my', 'your', 'arrang']}

In [175]:
score_word=[]              #Calculate scores 
for i in prueba['word']:
    if i in lmdict['Negative']:
        score_word.append('-1')    #If negative word, append -1
    elif i in lmdict['Positive']:
        score_word.append('1')     #If positive word, append 1
    else: score_word.append('0')  
        
prueba['score']= score_word

c:\users\marta\.virtualenvs\project-gaeps2at\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [176]:
prueba['score']=pd.to_numeric(prueba['score'])
prueba['score']= prueba['score']*prueba['count']   #Multiply score by count

c:\users\marta\.virtualenvs\project-gaeps2at\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\marta\.virtualenvs\project-gaeps2at\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [177]:
positive_rows= prueba[prueba['score']>0]
positive_rows

,track_id,word,count,score
63,TRAAAAV128F421A322,arrang,1,1
71,TRAAABD128F429CF47,and,2,2
77,TRAAABD128F429CF47,my,4,4
79,TRAAABD128F429CF47,your,7,7


In [178]:
negative_rows= prueba[prueba['score']<0]
negative_rows

,track_id,word,count,score
60,TRAAAAV128F421A322,slice,1,-1
66,TRAAAAV128F421A322,devast,1,-1
67,TRAAAAV128F421A322,element,1,-1


In [179]:
positive_rows= pd.DataFrame(positive_rows.groupby('track_id')['score'].sum())    #Sum of positive words in each song

In [180]:
negative_rows= pd.DataFrame(negative_rows.groupby('track_id')['score'].sum())    #Sum of negative words in each song

In [181]:
positive_rows

,score
track_id,
TRAAAAV128F421A322,1
TRAAABD128F429CF47,13


In [182]:
negative_rows

,score
track_id,
TRAAAAV128F421A322,-3


In [164]:
final_score= positive_rows.merge(negative_rows, on='track_id', how='left')
final_score.columns=['positive', 'negative']
final_score= final_score.fillna(0)
final_score['polarity']= (final_score['positive']-final_score['negative'])/(final_score['positive']+final_score['negative'])

In [183]:
final_score

,polarity
track_id,
TRAAAAV128F421A322,-2.0
TRAAABD128F429CF47,1.0


In [171]:
final_score=pd.DataFrame(final_score.loc[:,['polarity']])
final_score

,polarity
track_id,
TRAAAAV128F421A322,-2.0
TRAAABD128F429CF47,1.0


In [ ]:
#Now we need to pick a threshold: either 0> or the mean or median (or some more advanced methods)

In [ ]:
#Some preprocessing we could do

In [ ]:
# #Clean text
# replace_nospace= re.compile("[.;:!\'?,\"()\[\]]")
# replace_space= re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

# #Preprocess text
# def preprocess(text):
#     text=[replace_nospace.sub("", line.lower()) for line in text]
#     text=[replace_space.sub("", line) for line in text]
#     return text

# text=preprocess(text)

# #Remove stop words

# english_stop_words= stopwords.words('english')

# def remove_stop_words(corpus):
#     removed_stop_words=[]
#     for review in corpus:
#         removed_stop_words.append(' '.join([word for word in review.split() 
#                                            if word not in english_stop_words]))
#     return removed_stop_words

# text= remove_stop_words(text)

# df['text']= text

In [ ]:
#Now the question is, how to improve our dictionaries??
#Can we do it manually or by joining another dictionary? Can we add emphasizers or uncertain words?